In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Importing CRUD Python module KG
from pymongo import MongoClient
from animalshelter import AnimalShelter 

###########################
# Data Manipulation / Model
###########################

# Verifying credentials and CRUD module name for connection to database KG
username = "aacuser"
password = "SNHU1234"
animalshelter = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(animalshelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)

# Dropping '_id' column and removing it from dataframe KG
if '_id' in df.columns:
    df.drop(columns=['_id'],inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Kat Gilliland - Project Two')


# Adding Grazioso Salvare logo KG
image_filename = 'Grazioso Salvare Logo.png' # Adding Grazioso Salvare logo KG
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

# Setting up layout for Dash app KG
app.layout = html.Div(style={'backgroundColor': 'white'}, children=[ 
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center( # Adding Grazioso Salvare logo to center of webpage, required web link included KG
        html.A( 
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '50%', 'height': 'auto'}), # Placing HTML image tag KG
            href='www.snhu.edu'
        )
    ),
    html.Center(html.B(html.H1('Grazioso Salvare Dashboard by Kat Gilliland', style={'color':'slateblue'}))), # Centered header text with unique identifier KG
    html.Hr(),
    html.Div(style={'backgroundColor': 'pink', 'padding': '20px'}, children=[ # Giving the radio buttons/search fitlers some style KG
            dcc.RadioItems( # Setting radio buttons 
                id='filter_type',
                options=[
                    {'label': 'Water Rescue', 'value': 'Water Rescue'},
                    {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                    {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster Rescue or Individual Tracking'},
                    {'label': 'Reset', 'value': 'Reset'}
                ],
                value='Water Rescue', # Default value KG
                labelStyle={'display': 'inline-block', 'color': 'crimson'}) # Giving the text a little pop of color KG
    ]),     
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    # Adding in code for interactive filtering options KG
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[
                             {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
                         ],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
                         
                         # Making interactive data table user-friendly for the client KG
                         row_selectable = "single", # Enforcing single row selection for map callback KG
                         column_selectable = False, # User cannot select columns KG
                         
                         page_action = "native", # Only necessary data pulled from server, increases readability KG
                         page_current = 0, # Setting starting page at 0 KG
                         page_size = 10, # Setting 10 data outputs per page KG
                         sort_mode = "multi", # Client can sort through data in ascending or descending order KG
                         sort_action = "native", # Client can sort through data KG
                         filter_action = "native", # Client can filter through data KG
                         editable = False, # Client cannot edit cells KG
                         row_deletable = False, # Client cannot delete rows KG
                         selected_columns=[], # No columns initially set KG
                         selected_rows=[0] # First row is selected by default KG
                        ),
                html.Br(),
                html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
                html.Div(className='row',
                         style={'display' : 'flex'},
                         children=[
                             html.Div(
                                 id='graph-id',
                                 className='col s12 m6',
                             ),
                             html.Div(
                                 id='map-id',
                                 className='col s12 m6',
                             ),
                             html.H4("Kat Gilliland")
                         ])
                ])
    
#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    [Output('datatable-id','data'),
     Output('datatable-id', 'columns')],
    [Input('filter_type', 'value')])
def update_dashboard(filter_type):
## FIX ME Add code to filter interactive data table with MongoDB queries

# Filtering interactive data table KG
    if filter_type == 'Water Rescue':
        df_filtered = pd.DataFrame(list(animalshelter.read({'$and': [{'sex_upon_outcome' : 'Intact Female'},
                                                            {'$or': [
                                                                {'breed': 'Labrador Retriever Mix'},
                                                                {'breed': 'Chesapeake Bay Retriever'},
                                                                {'breed': 'Labrador Retriever/Newfoundland'},
                                                                {'breed': 'Newfoundland Mix'},
                                                                {'breed': 'Newfoundland/Australian Cattle Dog'},
                                                                {'breed': 'Newfoundland/Great Pyranees'},
                                                                {'breed': 'Newfoundland/Labrador Retriever'}]
                                                            },
                                                            {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                      {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                            }]
                                                  })))
    elif filter_type == 'Mountain or Wilderness Rescue':
        df_filtered = pd.DataFrame(list(animalshelter.read({'$and': [{'sex_upon_outcome' : 'Intact Male'},
                                                            {'$or': [
                                                                {'breed': 'German Shepherd'},
                                                                {'breed': 'Alaskan Malamute'},
                                                                {'breed': 'Old English Sheepdog'},
                                                                {'breed': 'Siberian Husky'},
                                                                {'breed': 'Rottweiler'}]
                                                            },
                                                            {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                      {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                            }]
                                                  })))
    elif filter_type == 'Disaster Rescue or Individual Tracking':
        df_filtered = pd.DataFrame(list(animalshelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                            {'$or': [
                                                                {'breed': 'Doberman Pinscher'},
                                                                {'breed': 'German Shepherd'},
                                                                {'breed': 'Golden Retriever'},
                                                                {'breed': 'Bloodhound'},
                                                                {'breed': 'Rottweiler'}]
                                                            },
                                                            {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 20}},
                                                                      {'age_upon_outcome_in_weeks': {'$lte': 300}}]
                                                            }]
                                                  })))
    elif filter_type == 'Reset':
        df_filtered = pd.DataFrame(list(animalshelter.read({}))) # When reset option is selected, all queries are read as search is reset KG
    
    if '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'], inplace=True)
    
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df_filtered.columns] # Columns cannot be deleted but they may be selected KG
    data = df_filtered.to_dict('records')
        
    return data, columns
            
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) 
    dff = pd.DataFrame.from_dict(viewData)
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    
    
    
    return [
        dcc.Graph(
            figure = px.pie( # Creating pie chart KG
                data_frame = dff,
                values = values,
                names = names,
                title = 'Available Breeds', # Descriptor for pie chart KG
                color_discrete_sequence=px.colors.sequential.RdBu,
                hole = 0.4, # Putting a hole in pie chart for aesthetics KG
            ),
            style={'width': '800px', 'height': '500px'}
        )
    ]
# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "selected_rows"),
     Input('datatable-id', "selected_columns")])
    
def update_map(viewData, selected_rows, selected_columns):  

    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:
        return []
    # Because we only allow single row selection, the list can be converted to a row index here
    if selected_rows == []:
        selected_rows = [0]
        
    # Austin TX is at [30.75,-97.48]
    if len(selected_rows) == 1:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
                dl.Marker(position=[dff.iloc[selected_rows[0],13],dff.iloc[selected_rows[0],14]], children=[
                    dl.Tooltip(dff.iloc[selected_rows[0],4]),
                    dl.Popup([
                    html.H4("Animal Name"),
                    html.P(dff.iloc[selected_rows[0],9]), # Column 9 defines animal name KG
                    html.H4("Sex"),
                    html.P(dff.iloc[selected_rows[0],12]), # Column 12 defines sex KG
                    html.H4("breed"),
                    html.P(dff.iloc[selected_rows[0],4]), # Column 4 defines breed KG
                    html.H4("Age"),
                    html.P(dff.iloc[selected_rows[0],15]) # Column 15 defines age KG
        
                ])
            ])
        ])
    ]

# Run program/execute code! KG
app.run(port=8052)  

 * Serving Flask app "Kat Gilliland - Project Two" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8052/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Apr/2024 18:55:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2024 18:55:52] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2024 18:55:53] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2024 18:55:53] "GET /_favicon.ico?v=2.8.1 HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2024 18:55:53] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [21/Apr/2024 18:55:53] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [21/Apr/2024 18:55:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2024 18:55:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2024 18:55:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2024 18:55:53] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [21/Apr/2024 18:55:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0